In [ ]:
# !wget https://snap.stanford.edu/data/roadNet-PA.txt.gz
# !gzip -d roadNet-PA.txt.gz
# !head -n10 roadNet-PA.txt

--2020-10-28 18:23:24--  https://snap.stanford.edu/data/roadNet-PA.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9945340 (9.5M) [application/x-gzip]
Saving to: ‘roadNet-PA.txt.gz’

roadNet-PA.txt.gz   100%[===================>]   9.48M  3.50MB/s    in 2.7s    

2020-10-28 18:23:27 (3.50 MB/s) - ‘roadNet-PA.txt.gz’ saved [9945340/9945340]



## Read edges

In [6]:
num_nodes = 1090920
num_edges = 3083796
# change k to generate different size of table
input_k = 32768

edges = []
for line in open('roadNet-PA.txt'):
    if line[0] != '#':
        u, v = line.strip().split('\t')
        u = int(u)
        v = int(v)
        edges.append((u, v))

print("raw data has #edges: {}".format(len(edges)))
print("raw data has #nodes: {}".format(max([max(u, v) for u, v in edges])))

raw data has #edges: 3083796
raw data has #nodes: 1090919


## Find the largest connected component

In [7]:
from collections import deque, Counter

def largest_wcc(num_nodes, edges):
    visited = [-1 for _ in range(num_nodes)]
    neighbors = [[] for _ in range(num_nodes)]
    for u, v in edges:
        neighbors[u].append(v)
        neighbors[v].append(u)

    # BFS
    num_wcc = 0
    for start in range(num_nodes):
        if visited[start] >= 0:
            continue

        visited[start] = num_wcc
        queue = deque([start])
        while len(queue) > 0:
            u = queue.popleft()
            for v in neighbors[u]:
                if visited[v] < 0:
                    visited[v] = num_wcc
                    queue.append(v)
        num_wcc += 1
  
    wcc, size = Counter(visited).most_common(1)[0]
    print(wcc, size)
    return [i for i in range(num_nodes) if visited[i] == wcc]

# selected_nodes contain the node indices of the connected component
selected_nodes = largest_wcc(num_nodes, edges)

0 1087562


## Shrink the graph by randomly merging the edges

In [8]:
import random

def get_parent(parent, node):
    visited = [node]
    while parent[node] != node:
        node = parent[node]
        visited.append(node)

    for n in visited:
        parent[n] = node
    return node


def random_shrink(N, selected_nodes, edges, new_N=2048):
    """Randomly shrink a graph.

    Args:
      N (int): the number of nodes in the graph
      selected_nodes (List of int): the connected component that we want to shrink
      edges (List of tuple): the edges of the graph
      new_N (int): the number of nodes that we want to keep

    Returns:
      List of int: the node indices after shrinking
      List of tuple: the remaining edges after shrinking
    """
    parent = list(range(N))

    current_N = len(selected_nodes)
    selected_nodes_set = set(selected_nodes)
    random.shuffle(edges)

    for u, v in edges:
        if u not in selected_nodes_set or \
           v not in selected_nodes_set:
            continue
        u_parent = get_parent(parent, u)
        v_parent = get_parent(parent, v)
        if u_parent != v_parent:
            parent[u_parent] = v_parent
            current_N -= 1
        if current_N <= new_N:
            break

    new_nodes = set([])
    for n in selected_nodes:
        new_nodes.add(get_parent(parent, n))
    new_nodes = set(new_nodes)

    new_edges = []
    for u, v in edges:
        if u not in selected_nodes_set or \
           v not in selected_nodes_set:
            continue

        u_parent = parent[u]
        v_parent = parent[v]
        if u_parent < v_parent:
            new_edges.append((u_parent, v_parent))
        elif u_parent > v_parent:
            new_edges.append((v_parent, u_parent))
    
    return new_nodes, new_edges

# new_nodes, new_edges = random_shrink(num_nodes, selected_nodes, edges, new_N=1024)
new_nodes, new_edges = random_shrink(num_nodes, selected_nodes, edges, input_k)

print("After shrinking, #nodes: {}\t#edges: {}".format(len(new_nodes), len(new_edges)))

After shrinking, #nodes: 32768	#edges: 82070


In [9]:
# reorder the shrinked network for node id and edges
def reorder_network(new_nodes, new_edges):
    map_dict = {} # mapping of node indices after reordering
    new_edges_reordered = []
    
    for counter, val in enumerate(set(new_nodes)):
        map_dict[val] = counter
        
    temp = {i: j for j, i in enumerate(set(new_nodes))} 
    new_nodes_reordered = [temp[i] for i in new_nodes]
    
    # reorder edges from map_dict
    for t in new_edges:
        lst = list(t) # tuple is immutable
        lst[0], lst[1] = map_dict[lst[0]], map_dict[lst[1]]
        new_edges_reordered.append(tuple(lst))
    return new_nodes_reordered, new_edges_reordered

def computeOutdegree(edges):
    outdegree = {}
    
    for t in edges:
        outdegree[t[0]] = outdegree[t[0]]+1 if (t[0] in outdegree) else 1
        
    return outdegree

def createNodeTable(nodes, k):
    output_node_table = "node_"+str(k)+".tbl"
    
    with open(output_node_table, 'w') as f:
        for n in nodes:
            f.write(str(n)+'|'+'\n')
    f.close()
    print("Finished generating node_{}.tbl".format(k))

def createEdgeTable(edges, k):
    output_edge_table = "edge_"+str(k)+".tbl"
    
    with open(output_edge_table, 'w') as f:
        for t in edges:
            f.write(str(t[0])+'|'+str(t[1])+'|'+str(1)+'|'+'\n')
                
        f.close()
        print("Finished generating edge_{}.tbl".format(k))
        
def createOutdegreeTable(nodes, edges, k):
    output_outdegree_table = "outdegree_"+str(k)+".tbl"
    # calculate outdegree due to our design schema in edge table
    outdegree = computeOutdegree(edges)
    
    with open(output_outdegree_table, 'w') as f:
        for n in nodes:
            # FIXME: all outdegree init to 1 to avoid divide by 0 when executing query
            degree = outdegree[n]+1 if (n in outdegree) else 1
            f.write(str(n)+'|'+str(degree)+'|'+'\n')
        f.close()
        print("Finished generating outdegree_{}.tbl".format(k))

def createPagerankTable(node_tbl, outdegree_tbl, k, alpha=0.85):
    output_pagerank_table = "pagerank_"+str(k)+".tbl"
    
    import re
    node_dict = {}
    with open(node_tbl) as f:
        for line in f:
            n = int(re.search(r'\d+', line).group(0))
            if n not in node_dict:
                node_dict[n] = 0
                
    initVal = (1-alpha)/len(node_dict)
    output = open(output_pagerank_table, 'w')
    with open(outdegree_tbl) as f:
        for line in f:
            id_, degree = re.findall(r'\d+', line)
            if int(id_) in node_dict:
                output.write(id_+'|'+"{:.9f}".format(initVal)+'|'+'\n')
        output.close()
        f.close()
    print("Finished generating pagerank_{}.tbl".format(k))

In [10]:
nodes_, edges_ = reorder_network(new_nodes, new_edges)
k = input_k
print("K is {}".format(k))
createNodeTable(nodes_, k)
createEdgeTable(edges_, k)
createOutdegreeTable(nodes_, edges_, k)
# must create node table and outdegree table first
createPagerankTable('node_'+ str(k) +'.tbl', 'outdegree_'+ str(k) +'.tbl',k)

K is 32768
Finished generating node_32768.tbl
Finished generating edge_32768.tbl
Finished generating outdegree_32768.tbl
Finished generating pagerank_32768.tbl
